# Developing with Python real time application user guide
In this guide we enumerate problems encountered while developing Real Time (RT) Application using python, and for each problem we present the suggested solution 
Lets start first by importing necessary libraries:

In [11]:
import sys
import os
import signal
import subprocess
import logging
from uuid import getnode, uuid4
from abc import ABC, abstractmethod
import time
import gc
import atexit
from functools import wraps

log = logging.getLogger(__name__)



The first problem we encountered while developing a RT application using Python is the garbage collector (GC). Python’s stop-the-world garbage collector makes latency non deterministic. When python decides it needs to run the garbage collector, the program gets stopped until it finishes. we will use the design pattern Decorator to inject to dismiss the GC before running critical section: First we call gc.disable() function, after that we add gc.set_threshold(0), the second instruction will prevent any third-party libraries calling gc.enable() during execution time, and so no libraries can brought the GC back. At The end of Node execution, we want the GC to clean up the memory, so we will register “os._exit” function throw the Decorator like this:

In [12]:
def gc_decorator(critical_func):
  
    @wraps(critical_func)
    def wrapper(*args, **kwargs):
        gc.disable()
        # gc.disable() doesn't work, because some random 3rd-party library will
        gc.set_threshold(0)
        # Suicide immediately after other atexit functions finishes.
        # CPython will do a bunch of cleanups in Py_Finalize which
        # will again cause Copy-on-Write, including a final GC
        atexit.register(os._exit, 0)
        return critical_func(*args, **kwargs)
    return wrapper

The second category of problem are :
* Device I/O: It is essential that real-time processes have all their memory kept in physical RAM and not paged out to swap.
* Memory management: Proper memory management is critical for real-time performance. In general, the programmer should avoid page faults in the real-time code path. During a page fault, the CPU pauses all computation and loads the missing page from disk into RAM (or cache, or registers). Loading data from disk is a slow and unpredictable operation. 

For memory management problem, variable declaration must be done at the initialization of the BaseNode class, and before any critical section, also using slot to declare python object can allow as control memory allocated and prevent adding additional attribute to an object later. To avoid any device IO problem, retrieving or writing data must be done outside the critical section.
At last for Exception problem: the throw is typically banned in hard real-time applications. Instead, we may rely on return codes to do error handling.
Lets see how can look our base class:

In [13]:
class BaseNode(ABC):
    __slots__ = ['_platform_id', '_pid', '_uid', ]  # using slot to declare python object can allow as
    # control memory allocated and prevent adding additional attribute to an object later

    def __init__(self, name=None):
        super().__init__()      
        self._platform_id = getnode()
        self._pid = os.getpid()
        self._uid = '%s#%s@%s' % (
                str(uuid4()),
                __name__,
                ''.join(("%012X" % self._platform_id)[i:i + 2] for i in range(0, 12, 2))
            )
    
    @abstractmethod
    def init(self):
        """
        To avoid any device IO problem, retrieving data must be done outside the critical section;
        All initialization, loading data from disk or similar tasks must be done here
        :return:
        """
        log.debug('Initializing Node ...')
        return 0

    @abstractmethod
    def run(self):
        """
        Code to be executed by the Node
        :return:
        """
        log.debug('starting execution of node ...')
        return 0

    @abstractmethod
    def finalize(self):
        """
        To avoid any device IO problem, writing data must be done outside the critical section;
        All finalization, writing data to disk or similar tasks must be done here
        :return:
        """
        log.debug('Finalizing the Node ...')
        return 0
            
    @gc_decorator
    def _execute(self):
        self.init()
        self.run()
        self.finalize()
        
class RTNode(BaseNode):
    def init(self):
        return 0
    
    def run(self):
        print("Node running.....")
        return 0
    
    def finalize(self):
        return 0
        

now to run the RT node we can do like this:

In [14]:
mynode = RTNode()
mynode._execute()

Node running.....


the third category of problems we can face when developing a RT application are:
* Latency: We must guarantee that our process doesn’t be pre-empted while dealing with a critical event, and we must also guarantee the latency of a process. 
* Global Interpreter Lock (GIL): Another notorious problem (at least for the standard CPython implementation of Python) is the GIL. The GIL prevent multiple python thread execute same code at the same time, so there are one GIL for each interpreter, allowing reference count being changed concurrently, the down side is in Python program, no matter how many threads exist, only one thread will be executed at a time, which means any attempt at using multithreading in order to gain concurrency benefits will be futile.
* Copy-on-Write (CoW): Linux kernel has a mechanism called CoW that serves as an optimization for forked processes. A child process starts by sharing every memory page with its parent. A page copied to the child’s memory space only when the page is written to, because of reference counting, every time we read a Python object, the interpreter will increase its refcount, which is essentially a write to its underlying data structure. This causes CoW and leads to page faults.
The proposed solutions for each problem are:
* Solving the latency problem: At the constructor level, we will indicate if yes or no this task is real time, also it takes as optional parameters: the scheduling policy and the priority, we can get the Node process name throw __file__ attribute, and after that use the subprocess.check_output to retrieve the process ID, setting the priority and the scheduling policy.
* To avoid the GIL problem, each actuator or sensor will be presented in COPDAI as Node. So, we will have just one process per task, and no need to use multithreading, against we will use multiprocessing, and so avoid the GIL downside
* To avoid the CoW problem, we will not use the default python multiprocessing library, despite each node must be in a separate file, and we will lunch each file by calling subprocess.Popen(['python3','nodes.py'])
the complete code can be found here [nodes.py](nodes.py)

In [16]:
proc = subprocess.Popen(['python3','nodes.py'])
print("process running with pid ", proc.pid)

process running with pid  99370
